In [1]:
import os
import numpy as np
import pandas as pd
import gzip, json
import random
from keras.models import Model, Sequential
from keras.layers import *
from keras.models import load_model
import keras.backend as K
import keras.callbacks
from keras.utils import Sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [2]:
train_path = "training"

e_lines = []
f_lines = []
for f in os.listdir(train_path):
    if f.endswith(".e"):
        with open(os.path.join(train_path, f)) as fc:
            for line in fc:
                e_lines.append(line[:-1])
    elif f.endswith(".f"):
        with open(os.path.join(train_path, f)) as fc:
            for line in fc:
                f_lines.append(line[:-1])

In [3]:
idx = 42
print(e_lines[idx])
print(f_lines[idx])

Six seats have been added to reflect the increasing number of Canadian electors.  
Six nouveaux sièges ont été ajoutés à la Chambre des communes afin qu'elle reflète adéquatement le nombre croissant d'électeurs canadiens.  


In [4]:
for i, line in enumerate(e_lines):
    for c in ".,!?;:()\n_-'`«»%$":
        e_lines[i] = e_lines[i].replace(c, " " + c + " ")
for i, line in enumerate(f_lines):
    for c in ".,!?;:()\n_-'`«»%$":
        f_lines[i] = f_lines[i].replace(c, " " + c + " ")

In [6]:
idx = 42
print(e_lines[idx])
print(f_lines[idx])

Six seats have been added to reflect the increasing number of Canadian electors .   
Six nouveaux sièges ont été ajoutés à la Chambre des communes afin qu ' elle reflète adéquatement le nombre croissant d ' électeurs canadiens .   


In [7]:
e_vocab = set()
f_vocab = set()

for line in e_lines:
    for w in line.split(" "):
        if w not in e_vocab:
            e_vocab.add(w)
e_vocab = list(e_vocab)
for line in f_lines:
    for w in line.split(" "):
        if w not in f_vocab:
            f_vocab.add(w)
f_vocab = list(f_vocab)

In [8]:
print(len(e_vocab), e_vocab)

61718 ['', 'calamity', 'OC', 'sciage', 'Westerdam', 'status', 'sailed', 'Puisne', 'dizziness', 'Redditt', 'elusive', 'Immunization', 'Tassé', 'constuction', 'OBJECTORS', 'RMs', 'sombreros', 'twiddling', 'anthropology', 'prerogatives', 'placebo', 'Prisons', 'receiptable', 'COPA', 'quake', 'necks', 'Spud', 'COSTA', 'Katy', 'solemnisation', 'FOOT', 'limbs', 'FAMOUS', 'prorogation', '37th', 'describing', 'adequate', 'Participants', 'Allmand', 'potentiellement', 'PETITCODIAC', 'rodent', 'technology', 'Teachers', 'VÉZINA', 'motorhome', 'scientists', 'phoney', 'Benevolent', 'fables', 'CMA', 'implored', 'Isaac', 'fiascos', 'absorbed', 'aircrew', 'sales', 'Kowalchuk', 'utopia', 'commander', 'antisocial', 'Onshore', 'Prototype', 'patriation', 'papering', 'findings', 'ambiguous', 'Albert', 'parting', '3°', 'ICAO', '411', 'transcended', 'surges', 'evokes', 'Bellefeuille', 'Smaller', 'Extreme', 'TERRITORIES', 'reflecting', 'Properties', 'imputation', 'foreboding', 'Crying', 'mucous', 'relics', 'per

In [9]:
print(len(f_vocab), f_vocab)

77444 ['', 'frottera', 'protocoles', 'OC', 'sciage', 'définiront', 'Westerdam', 'compromettrions', 'jusque', 'Professeur', 'Tassé', 'Redditt', 'dépassés', 'réseauter', 'constuction', 'dictent', 'Entendons', 'Isobel', 'abandonnèrent', 'étaler', 'prescrive', 'canadianiser', 'sombreros', 'signalerez', 'ridiculisaient', 'remettrons', 'filtrée', 'verrouillèrent', 'abondent', 'accouplement', 'placebo', 'automatismes', 'apathiques', 'autodiscipline', 'valorisante', 'reconsidérera', 'Sortons', 'COPA', 'necks', 'COSTA', 'Katy', 'toxicomanogène', 'FAMOUS', 'prorogation', 'assistons', 'Ajouter', 'Allmand', 'pathologiques', 'ressentait', 'potentiellement', 'influerait', 'Contourner', 'PETITCODIAC', 'simulerait', 'attachement', 'technology', 'Teachers', 'VÉZINA', 'profilent', 'grandissant', 'captées', 'gobe', 'exclamée', 'Benevolent', 'lard', 'fables', 'enlisées', 'CMA', 'marqués', 'Isaac', 'limitatives', 'thermiques', 'chanterai', 'sales', 'Kowalchuk', 'pêchée', 'commander', 'antisocial', 'désacra

In [12]:
batch_size = 32
max_encoder_seq_len = max([len(line) for line in e_lines])
max_decoder_seq_len = max([len(line) for line in f_lines])
latent_dim = 256
num_encoder_tokens = len(e_vocab)
num_decoder_tokens = len(f_vocab)

class dataGenerator(Sequence):
    def __init__(self, x, y, batch_size):
        self.batch_size = batch_size
        self.x = x
        self.y = y
        
    def __len__(self):
        return (len(self.x) // self.batch_size) - 1
    
    def __getitem__(self, idx):
        X = []
        Y_start = []
        Y_end = []
        for i in range(self.batch_size):
            X = [to_categorical(vocab.index(w), num_classes=len(vocab)) for w in self.x[idx * batch_size + i]]
            Y_start = 
            Y_end = 
        return [np.array(X), np.array(Y_start)], np.array(Y_end)

split = int(0.8 * len(e_lines))    
data_gen = dataGenerator(e_lines[:split], f_lines[:split], batch_size)
val_gen = dataGenerator(e_lines[split:], f_lines[split:], batch_size)

SyntaxError: invalid syntax (<ipython-input-12-8b37da1e0cfc>, line 23)

In [11]:
K.clear_session()

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.summary()

NameError: name 'num_encoder_tokens' is not defined

In [26]:
#save best model if model improved
model_name = "translator_en_fr.h5"
best_checkpoint = keras.callbacks.ModelCheckpoint(model_name, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [27]:
h = model.fit_generator(data_gen, epochs = 30, callbacks=[best_checkpoint])

Epoch 1/30


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0....